# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [29]:
from pyspark.sql import SparkSession
import re
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler, MinMaxScaler


# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [5]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [6]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [7]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [19]:
# TODO: write your code to answer question 1
#df.filter("BodyLength").where(df.Id == "1112").show()
df.select("BodyLength").where(df.Id == "1112").show()


+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [20]:
# TODO: write your code to answer question 2
df = df.withColumn("titl_bdy", concat(col("Title"), lit(' '), col("Body")))

In [21]:
regexTokenizer = RegexTokenizer(inputCol="titl_bdy", outputCol="words_2", pattern="\\W")
df = regexTokenizer.transform(df)
df = df.withColumn("Desc_Length", body_length(df.words_2))

In [22]:
df.select("Desc_Length").where(df.Id == "5123").show()

+-----------+
|Desc_Length|
+-----------+
|        135|
+-----------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [23]:
# TODO: write your code to create this vector
assembler2 = VectorAssembler(inputCols=["Desc_Length"], outputCol="DescVec")
df = assembler2.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [24]:
# TODO: write your code to answer question 3
scaler_q1 = Normalizer(inputCol="DescVec", outputCol="DescVecNormalizer")
df = scaler_q1.transform(df)

In [25]:
df.select("DescVecNormalizer").where(df.Id == "512").show()

+-----------------+
|DescVecNormalizer|
+-----------------+
|            [1.0]|
+-----------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [38]:
# TODO: write your code to answer question 4
scaler_q2 = StandardScaler(inputCol="DescVec", outputCol="DescVecStandardScaler5", withMean=True, withStd=True)
scalerModel_q2 = scaler_q2.fit(df)
df = scalerModel_q2.transform(df)

In [39]:
df.select("DescVecStandardScaler5").where(df.Id == "512").show()

+----------------------+
|DescVecStandardScaler5|
+----------------------+
|  [-0.6417314460998...|
+----------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [33]:
scaler_q3 = MinMaxScaler(inputCol="DescVec", outputCol="DescVecMinMAxScaler")
scalerModel_q3 = scaler_q3.fit(df)
df = scalerModel_q3.transform(df)

In [36]:
# TODO: write your code to answer question 5
df.select("DescVecMinMAxScaler").where(df.Id == "512").show()

+--------------------+
| DescVecMinMAxScaler|
+--------------------+
|[0.00624833820792...|
+--------------------+

